In [7]:
# import lib
# ===========================================================
import csv
import pandas as pd
from datascience import *
import numpy as np
import random
import time
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')
import collections
import math
import sys
from tqdm import tqdm
from time import sleep

In [40]:
# Initialize useful data
# with open('clinvar_conflicting_clean.csv', 'r') as f:
#     reader = csv.reader(f)
#     temp_rows = list(reader)
df = pd.read_csv('clinvar_conflicting_clean.csv', low_memory=False)
columns_to_change = ['ORIGIN', 'EXON', 'INTRON', 'STRAND', 'LoFtool', 'CADD_PHRED', 'CADD_RAW', 'BLOSUM62']
df[['CLNVI', 'MC', 'SYMBOL', 'Feature_type', 'Feature', 'BIOTYPE', 
 'cDNA_position', 'CDS_position', 'Protein_position', 'Amino_acids', 'Codons', 
 'BAM_EDIT', 'SIFT', 'PolyPhen']] = df[['CLNVI', 'MC', 'SYMBOL', 'Feature_type', 'Feature', 'BIOTYPE', 
 'cDNA_position', 'CDS_position', 'Protein_position', 'Amino_acids', 'Codons', 
 'BAM_EDIT', 'SIFT', 'PolyPhen']].fillna(value="null")
# df_zero = df.loc[df['CLASS'] == 0]
# df_zero = df_zero.sample(n=10000)
# df_one = df.loc[df['CLASS'] == 1]
# df_one = df_one.sample(n=10000)

# df = pd.concat([df_zero, df_one])
# df = df.sample(n = df.shape[0])
all_rows = df.values.tolist()
row_num = len(all_rows)
df.head()

,CHROM,POS,REF,ALT,AF_ESP,AF_EXAC,AF_TGP,CLNDISDB,CLNDN,CLNHGVS,...,Codons,STRAND,BAM_EDIT,SIFT,PolyPhen,LoFtool,CADD_PHRED,CADD_RAW,BLOSUM62,CLASS
0,1,955563,G,C,0.0000,0.00000,0.0000,"MedGen:C3808739,OMIM:615120|MedGen:CN169374","Myasthenic_syndrome,_congenital,_8|not_specified",NC_000001.10:g.955563G>C,...,cGg/cCg,1.0,null,null,null,0.421,11.390,1.133255,-2.0,0
1,1,955597,G,T,0.0000,0.42418,0.2826,MedGen:CN169374,not_specified,NC_000001.10:g.955597G>T,...,ccG/ccT,1.0,null,null,null,0.421,8.150,0.599088,NaN,0
2,1,955619,G,C,0.0000,0.03475,0.0088,"MedGen:C3808739,OMIM:615120|MedGen:CN169374","Myasthenic_syndrome,_congenital,_8|not_specified",NC_000001.10:g.955619G>C,...,Gtc/Ctc,1.0,null,null,null,0.421,3.288,0.069819,1.0,1
3,1,957640,C,T,0.0318,0.02016,0.0328,"MedGen:C3808739,OMIM:615120|MedGen:CN169374","Myasthenic_syndrome,_congenital,_8|not_specified",NC_000001.10:g.957640C>T,...,gaC/gaT,1.0,null,null,null,0.421,12.560,1.356499,NaN,0
4,1,976059,C,T,0.0000,0.00022,0.0010,MedGen:CN169374,not_specified,NC_000001.10:g.976059C>T,...,Ctg/Ttg,1.0,null,null,null,0.421,17.740,2.234711,NaN,1


In [41]:
# Decision stump part for Adaboost
# ===========================================================
def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)

# === LeafNode is the prediction result of this branch ===
class LeafNode:
    def __init__(self, rows):
#         labels = [row[-1] for row in rows]
        labels = []
        self.one_idx = []
        self.zero_idx = []
        for i in range(len(rows)):
            row = rows[i]
            labels.append(row[-1])
            if row[-1] == 1:
                self.one_idx.append(i)
            else:
                self.zero_idx.append(i)
        self.prediction = collections.Counter(labels)
        
# === DecisionNode is an attribute / question used to partition the data ===
class DecisionNode:
    def __init__(self, question = None, left_branch = None, right_branch = None):
        self.question = question
        self.left_branch = left_branch
        self.right_branch = right_branch
    
class DecisionStump:
    def __init__(self, training_attribute, training_data, height, method = "CART"):
        self.attribute = training_attribute     # takein attribute and data separately
        self.train = training_data
        self.height = height
        self.row_num = len(self.train)
        self.column_num = len(self.attribute)
        self.method = method.upper()            # convert to upper case for general use
        self.labels = self.uniq_val(-1)
        self.significance = 0
        if self.method not in ["C4.5", "CART", "HYBRID"]:
            print("Error: Please choose a valid method! from: [C4.5, CART, HYBRID]")
            return None
        
        # train decision stump
        self.root = self.build_stump(self.train, 1)
        
        # count ACC classifications and mis classifications to update weights
        self.accclassify_idx = []
        self.misclassify_idx = []
        # walk down the decision stump to collect all misclassification indices
        
#         if self.root.left_branch.prediction.get(1, 0) > self.root.left_branch.prediction.get(0, 0):
#             # then consider the prediction of this leaf node as 1: 1 -> correct, 0 -> misclassify
#             self.accclassify_idx += self.root.left_branch.one_idx
#             self.misclassify_idx += self.root.left_branch.zero_idx
#         else:
#             # then consider the prediction of this leaf node as 0: 0 -> correct, 1 -> misclassify
#             self.accclassify_idx += self.root.left_branch.zero_idx
#             self.misclassify_idx += self.root.left_branch.one_idx
#         if self.root.right_branch.prediction.get(1, 0) > self.root.right_branch.prediction.get(0, 0):
#             # then consider the prediction of this leaf node as 1: 1 -> correct, 0 -> misclassify
#             self.accclassify_idx += self.root.right_branch.one_idx
#             self.misclassify_idx += self.root.right_branch.zero_idx
#         else:
#             # then consider the prediction of this leaf node as 0: 0 -> correct, 1 -> misclassify
#             self.accclassify_idx += self.root.right_branch.zero_idx
#             self.misclassify_idx += self.root.right_branch.one_idx
    
    def uniq_val(self, column):
        return set([self.train[i][column] for i in range(len(self.train))])
    
    # when raising a question.
    # if it's a categorical attribute, we simply iterate all categories
    # if it's a numeric attribute, we iterate the set of possible numeric values 
    class Question:
        def __init__(self, column, ref_value, attribute):
            self.column = column
            self.ref_value = ref_value if ref_value else "None"
            self.attri = attribute

        def match(self, row):
            if is_numeric(self.ref_value):
                try:
                    return row[self.column] >= self.ref_value
                except:
                    print("Error occured in ", row)
                    return True
            else:
                return row[self.column] == self.ref_value

        def __repr__(self):
            operand = ">=" if is_numeric(self.ref_value) else "=="
            return "Is %s %s %s?" % (self.attri[self.column], operand, str(self.ref_value))
    
    # === Method 1 - C4.5 ===
    def entropy(self, rows):
        # === Bits used to store the information ===
        labels = [row[-1] for row in rows]
        frequency = collections.Counter(labels).values()
        pop = sum(frequency)
        H = 0
        for f in frequency:
            p = f / pop
            H -= p * math.log(p, 2)
        return H
    
    # === Method 2 - CART ===
    def gini(self, rows):
        # === Probability of misclassifying any of your label, which is impurity ===
        labels = [row[-1] for row in rows]
        frequency = collections.Counter(labels).values()
        pop = sum(frequency)
        gini = 1
        for f in frequency:
            p = f / pop
            gini -= p ** 2
        return gini
    
    # === Calculate Gain Info ===
    def info(self, branches, root):
        # === Objective: to find the best question which can maximize info ===
        root_size = float(len(root))
        if self.method == "C4.5":  # Here I pick the GainRatio Approach
            root_uncertainty = self.entropy(root)
            gain_info = root_uncertainty
            split_info = 0
            for branch in branches:
                if not branch: continue
                gain_info -= len(branch) / root_size * self.entropy(branch)
                split_info -= float(len(branch)) / root_size * math.log(float(len(branch)) / root_size)
#                 print(gain_info, split_info)
            return gain_info / split_info
        elif self.method == "CART":
            root_uncertainty = self.gini(root)
            gain_info = root_uncertainty
            for branch in branches:
                if not branch: continue
                gain_info -= len(branch) / root_size * self.gini(branch)
            return gain_info
        elif self.method == "HYBRID":
            pass
        pass
    
    # === Here I only do Binary Partitions ===
    def partition(self, rows, question):
        true_rows = []
        false_rows = []
        for row in rows:
            if question.match(row):
                true_rows.append(row)
            else:
                false_rows.append(row)
        return true_rows, false_rows
    
    # the question that achieves the max infomation attenuation is the best question
    def find_best_question(self, rows):
        max_info_attenuation = 0
        best_question = self.Question(0, self.train[0][0], self.attribute)
        # === Iterate through all question candidates ===
        # === TODO: Maybe Iteration here can be optimized ===
        for col in range(self.column_num - 1): # minus 1 to avoid using the label as attribute
            ref_candidates = self.uniq_val(col)
            for ref_value in ref_candidates:
                if ref_value == "null": continue # avoid using null values to generate a question
                q = self.Question(col, ref_value, self.attribute)
                temp_true_rows, temp_false_rows = self.partition(rows, q)
                temp_info_attenuation = self.info([temp_true_rows, temp_false_rows], rows)
                if temp_info_attenuation >= max_info_attenuation:
                    max_info_attenuation = temp_info_attenuation
                    best_question = q
        return max_info_attenuation, best_question
        
    # === Input rows of data with attributes and labels ===
    def build_stump(self, rows, height):
        # === Assign all rows as root of the whole decision tree ===
        # === We have met the leaf node if gini(rows) is 0 or no question candidates left ===
        gain_reduction, q = self.find_best_question(rows)
        true_rows, false_rows = self.partition(rows, q)
        if height == self.height:
            return DecisionNode(q, LeafNode(true_rows), LeafNode(false_rows))
        else:
            return DecisionNode(q, self.build_tree(true_rows, height + 1), self.build_tree(false_rows, height + 1))
    
    # === Input a row of data with attributes (and no label), predict its label with our decision tree ===
    # === Actually it can contain a label, we just don't use it ===
    # === walk down the decision tree until we reach the leaf node ===
    def classify(self, row, node):
        if isinstance(node, LeafNode):
            # do a mapping from label[1, 0] to label[1, -1]
            return node.prediction
#             return 1 if node.prediction.get(1, 0) / (node.prediction.get(1, 0) + node.prediction.get(0, 0)) > cutoff else -1
        
        if node.question.match(row):
            return self.classify(row, node.left_branch)
        else:
            return self.classify(row, node.right_branch)

    # function to print the tree out
    def print_tree(self, node, spacing=""):
        # Base case: we've reached a leaf
        if isinstance(node, LeafNode):
            print (spacing + "Predict", node.prediction)
            return

        # Print the question at this node
        print (spacing + str(node.question))

        # Call this function recursively on the true branch
        print (spacing + '--> True:')
        self.print_tree(node.left_branch, spacing + "  ")

        # Call this function recursively on the false branch
        print (spacing + '--> False:')
        self.print_tree(node.right_branch, spacing + "  ")
    
    def test(self):
        for i in range(self.column_num):
            q = self.Question(i, self.train[1][i], self.attribute)
            print(q)
            print(q.match(1))
            
def normalized_weight(weight):
    return np.divide(weight, sum(weight))

def rev_logit(val):
    return 1 / (1 + np.exp(val))

In [42]:
# Divide whole dataset into training set and testing set
# ===========================================================
training_percentage = 0.01  # percent of partition of training dataset
training_size = int(row_num * training_percentage)
testing_size = row_num - training_size
training_attribute = list(df.columns)
training_data = all_rows[: training_size]  # training data should include header row
testing_data = all_rows[training_size: ]   # testing data don't need to include header row

In [43]:
# Recursively Training base learners
# ===========================================================
# let's train T = 10 base learners
T = 10
weakleaner_height = 3
stump_forest = []
weight = [1 / training_size for _ in range(training_size)]
start = time.time()
for i in range(T):
    # train a decision stump
    print("training stump %d" % i)
    stump = DecisionStump(training_attribute, training_data, height = weakleaner_height, "CART")
    # calculate the total error of the stump
    # TBD: maybe can be integrated into the training process
    accuracy = len(stump.accclassify_idx) / training_size
    
    total_err_rate = 1 - accuracy
    # update the significance level of this stump, remember not to divide by zero
    stump.significance = 0.5 * math.log((1 - total_err_rate + 0.0001) / (total_err_rate + 0.0001))
    # append stump into the forest
    stump_forest.append(stump)
    if len(stump_forest) == T: break  # early break
    # update training_data weight, resample the training data with the updated weight distribution
    true_scale = np.e ** stump.significance
    for idx in stump.misclassify_idx:
        weight[idx] = weight[idx] * true_scale
    for idx in stump.accclassify_idx:
        weight[idx] = weight[idx] * (1 / true_scale)
    distrib = normalized_weight(weight)
    print("stump %d trained, alpha = %.02f" % (i, stump.significance))
    resampled_idx = np.random.choice(training_size, training_size, p = distrib)
    training_data = [training_data[idx] for idx in resampled_idx]
    stump.print_tree(stump.root)
    weight = [1 / training_size for _ in range(training_size)]
end = time.time()
print(end - start, "seconds")

training stump 0
stump 0 trained, alpha = 0.62
Is AF_TGP >= 0.0076?
--> True:
  Predict Counter({0: 156, 1: 7})
--> False:
  Predict Counter({0: 349, 1: 139})
training stump 1
stump 1 trained, alpha = 0.79
Is CLNDN == Distal_spinal_muscular_atrophy|Distal_spinal_muscular_atrophy,_autosomal_recessive_4|Charcot-Marie-Tooth_disease,_recessive_intermediate_c?
--> True:
  Predict Counter({1: 18, 0: 1})
--> False:
  Predict Counter({0: 521, 1: 111})
training stump 2
stump 2 trained, alpha = 0.66
Is AF_TGP >= 0.006999999999999999?
--> True:
  Predict Counter({0: 191, 1: 4})
--> False:
  Predict Counter({0: 323, 1: 133})
training stump 3
stump 3 trained, alpha = 0.68
Is Consequence == inframe_insertion?
--> True:
  Predict Counter({1: 15})
--> False:
  Predict Counter({0: 504, 1: 132})
training stump 4
stump 4 trained, alpha = 0.68
Is Codons == Cgg/Tgg?
--> True:
  Predict Counter({1: 27, 0: 4})
--> False:
  Predict Counter({0: 491, 1: 129})
training stump 5
stump 5 trained, alpha = 0.76
Is Lo

In [44]:
# New Testing Adaboost
# TBD
# ===========================================================
# Compute TN, TP, FN, FP, etc. together with testing
# ===========================================================
ROC = Table(make_array('CUTOFF', 'TN', 'FN', 'FP', 'TP', 'ACC'))
step_size = 0.05
forest_size = len(stump_forest)
CMap = {0: 'TN', 1: 'FN', 2: 'FP', 3: 'TP'}
for cutoff in np.arange(0, 1 + step_size, step_size):
    Confusion = {'TN': 0, 'FN': 0, 'FP': 0, 'TP': 0}
    for row in testing_data:
        true_rate_forest = 0
        for tree_i in stump_forest:
            # prediction is a counter of label 1 and 0
            pred_counter = tree_i.classify(row, tree_i.root)
            # do a mapping from label[1, 0] to label[1, -1]
            true_rate_tree = 1 if pred_counter.get(1, 0) / (pred_counter.get(1, 0) + pred_counter.get(0, 0) + 0.00000001) > cutoff else -1
            true_rate_forest += true_rate_tree * tree_i.significance
#         true_rate_forest = rev_logit(true_rate_forest)
#         true_pred = 1 if true_rate_forest >= cutoff else 0
        true_pred = 0 if np.sign(true_rate_forest) <= 0 else 1
        indicator = (true_pred << 1) + row[-1]
        # accordingly update confusion matrix
        Confusion[CMap[indicator]] += 1
    # concatenate the confusion matrix values into the overall ROC Table
    thisline = [cutoff] + list(Confusion.values()) + [(Confusion['TP'] + Confusion['TN']) / sum(Confusion.values())]
    ROC = ROC.with_row(thisline)
ROC = ROC.with_columns('SENSITIVITY', ROC.apply(lambda TP, FN: TP / (TP + FN + 0.00000001), 'TP', 'FN'))
ROC = ROC.with_columns('FPR', ROC.apply(lambda TN, FP: FP / (TN + FP + 0.00000001), 'TN', 'FP'))
ROC = ROC.with_column('FMEAS', ROC.apply(lambda TP, FP, FN: 2 * (TP / (TP + FN)) * (TP / (TP + FP)) / (TP / (TP + FN) + TP / (TP + FP)), 'TP', 'FP', 'FN'))

/Users/frostace/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in long_scalars


In [45]:
ROC.show()

CUTOFF,TN,FN,FP,TP,ACC,SENSITIVITY,FPR,FMEAS
0,0,0,48249,16288,0.252382,1,1,0.403044
0.05,0,0,48249,16288,0.252382,1,1,0.403044
0.1,0,0,48249,16288,0.252382,1,1,0.403044
0.15,0,0,48249,16288,0.252382,1,1,0.403044
0.2,7099,497,41150,15791,0.35468,0.969487,0.852867,0.431277
0.25,48249,16288,0,0,0.747618,0,0,nan
0.3,48249,16288,0,0,0.747618,0,0,nan
0.35,48249,16288,0,0,0.747618,0,0,nan
0.4,48249,16288,0,0,0.747618,0,0,nan
0.45,48249,16288,0,0,0.747618,0,0,nan


In [ ]:
# Original Testing
# ===========================================================
accuracy = 0
for row in testing_data:
    overall_classification = 0
    for stump in stump_forest:
        classification = stump.classify(row, stump.root)
        vote = stump.significance
        overall_classification += classification * vote
    # reverse mapping from label[1, -1] to label[1, 0]
    predicted_label = 0 if np.sign(overall_classification) <= 0 else 1
    if predicted_label == row[-1]: accuracy += 1
#     print(classification, predicted_label, row[-1])
accuracy = accuracy / testing_size
print("%.03f%%" % (accuracy * 100))

In [ ]:
# Testing with a toy dataset
# ===========================================================
training_data = [
    ['Green', 3, 1],
    ['Yellow', 3, 1],
    ['Red', 1, 0],
    ['Red', 1, 0],
    ['Yellow', 3, 1],
    ['Red', 3, 1]
]
testing_data = [
    ['Red', 2, 0],
    ['Yellow', 3.5, 1],
    ['Green', 3, 1]
]
training_attribute = ['Color', 'Diameter', 'Label']
training_size = len(training_data)
testing_size = len(testing_data)

In [38]:
# pf = df[]
len(set(pd.Index(df)))

20000

In [29]:
np.bincount([row[-1] for row in all_rows])

array([10000, 10000])